In [94]:
import import_ipynb

In [95]:
!ls

Cleaned_data_FINAL.ipynb       Untitled.ipynb
Initial_EDA_beeradvocate.ipynb Untitled1.ipynb
Initial_EDA_beerdata.ipynb


In [96]:
import import_ipynb
import Initial_EDA_beeradvocate
import Initial_EDA_beerdata
import Untitled
from Untitled import *
from Initial_EDA_beeradvocate import *
from Initial_EDA_beerdata import *
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

We have a bunch of clean data sets now, we need to try and join these together.

The variable names we have are the following:

1. df_ba
2. df_beers
3. df_breweries - we're not using this dataset.
4. df_reviews



In [97]:
df_ba.head()
# df_ba.set_index(['beer_beerid'], inplace=True)

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [98]:
df_beers.head()

,id,name,brewery_id,state,country,style,availability,notes,retired
0,202522,Olde Cogitator,2199,CA,US,English Oatmeal Stout,Rotating,No notes at this time.,f
1,82352,Konrads Stout Russian Imperial Stout,18604,Not Applicable,NO,Russian Imperial Stout,Rotating,No notes at this time.,f
2,214879,Scottish Right,44306,IN,US,Scottish Ale,Year-round,No notes at this time.,t
3,320009,MegaMeow Imperial Stout,4378,WA,US,American Imperial Stout,Winter,Every time this year,f
4,246438,Peaches-N-Cream,44617,PA,US,American Cream Ale,Rotating,No notes at this time.,f


In [99]:
df_reviews.head()
# df_reviews.set_index(['beer_id'], inplace=True)

,beer_id,username,date,text,look,smell,taste,feel,overall,score
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of 360. Served in a DFH snifter, the beer pours dark brown/black with about an inch tan head that stuck around a little while. There's also a good amount of lacing. I like the aroma, the brew smells like dark fruit (raisin, plum), chocolate, roasted malt, and a bit of licorice. For the most part I think the taste is just like the aroma, but with some coffee flavor noticeable too. Mouthfeel/body is medium/full, it's creamy and coating with a moderate amount of carbonation. I think it's a good overall brew, worth picking up if you want a nice straight forward imperial stout. Here's hoping Four String brings this back in the future! $9.99 a bottle.",4.00,4.00,4.00,4.25,4.00,4.03
1,125646,_dirty_,2017-12-21,,4.50,4.50,4.50,4.50,4.50,4.50
2,125646,CJDUBYA,2017-12-21,,4.75,4.75,4.75,4.75,4.75,4.75
3,125646,GratefulBeerGuy,2017-12-20,"0% 16 oz can. Funny story: As I finally walked in the doors after a 45 min wait in line and freezing temps the sweet sound of the Grateful Dead's Sugar Magnolia greeted me from the TreeHouse sound system. The bottom of the can reads: ""Going where the wind goes, bloomin' like a red rose"" A white haze to the yellow and golden liquid. Thick and healthy, totally unfiltered. Brawny white foam cap, thick, all-white clumps. Huge lacing left over. The aroma has a very zesty citrus hop effect, mellon and mango, grainy earthiness, tropical fruit blend with a bitter to sweet effect, then a peppery kick at the end. Very aromatic. The flavor is just bursting with complex hops, zesty earthy tones, sweet orange, peppery malt, clean fresh feel and overall vibe. A crispy bite wakes you up, full and lush mouthfeel follows from a totally unfiltered expereince. The feel and flavor finishes with a fun, earthy, zesty dry bite. Tropical juicy, zesty citrus, zippy golden wheat malt, melons, rustic earthiness sums up the taste pretty well. Levels of complexity are deep. It's an interesting ride for sure. Overall, this one stands somewhere near the top of the New England IPA's.",4.75,4.75,4.50,4.50,4.50,4.58
4,125646,LukeGude,2017-12-20,Classic TH NEIPA. Overflowing head and bouquet. Strong flavor but smoother than 8.2%. Maybe peach nose but overall a general citrus punch.,4.25,4.50,4.25,4.25,4.25,4.31


In [100]:
# df_ba.join(df_reviews, how='left')

In [101]:
df_ny = df_beers[df_beers['country'] == 'US']
df_ny.set_index(['id'])

,name,brewery_id,state,country,style,availability,notes,retired
id,,,,,,,,
202522,Olde Cogitator,2199,CA,US,English Oatmeal Stout,Rotating,No notes at this time.,f
214879,Scottish Right,44306,IN,US,Scottish Ale,Year-round,No notes at this time.,t
320009,MegaMeow Imperial Stout,4378,WA,US,American Imperial Stout,Winter,Every time this year,f
246438,Peaches-N-Cream,44617,PA,US,American Cream Ale,Rotating,No notes at this time.,f
108605,Icon Sender,22598,CA,US,American Lager,Year-round,No notes at this time.,f
...,...,...,...,...,...,...,...,...
258269,Brewers Select #019 High Pressure Lager,28281,WA,US,American Lager,Rotating,No notes at this time.,f
146613,American Pale Ale,36926,WI,US,American Pale Ale (APA),Rotating,No notes at this time.,t
187618,RIPTA,34665,RI,US,Belgian Tripel,Rotating,No notes at this time.,f


In [102]:
df_beers['state'].value_counts()

Not Applicable    60572
CA                33649
PA                17083
NY                14572
CO                14182
                  ...  
MB                171  
PE                98   
YT                86   
NL                77   
NT                4    
Name: state, Length: 68, dtype: int64

In [103]:
df_reviews.value_counts()

beer_id  username         date        text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [104]:
df_ny.join(df_ba, how='left', lsuffix='_2')

,id,name,brewery_id_2,state,country,style,availability,notes,retired,brewery_id,...,review_time,review_overall,review_aroma,review_appearance,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,202522,Olde Cogitator,2199,CA,US,English Oatmeal Stout,Rotating,No notes at this time.,f,10325,...,1234817823,1.5,2.0,2.5,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
2,214879,Scottish Right,44306,IN,US,Scottish Ale,Year-round,No notes at this time.,t,10325,...,1235916604,3.0,2.5,3.0,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,320009,MegaMeow Imperial Stout,4378,WA,US,American Imperial Stout,Winter,Every time this year,f,10325,...,1234725145,3.0,3.0,3.5,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,246438,Peaches-N-Cream,44617,PA,US,American Cream Ale,Rotating,No notes at this time.,f,1075,...,1293735206,4.0,4.5,4.0,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
6,108605,Icon Sender,22598,CA,US,American Lager,Year-round,No notes at this time.,f,1075,...,1318991115,3.5,3.5,3.5,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358865,258269,Brewers Select #019 High Pressure Lager,28281,WA,US,American Lager,Rotating,No notes at this time.,f,1349,...,1257629853,4.0,2.5,3.0,Bock,3.5,3.5,Saaremaa Tume,6.5,51372
358867,146613,American Pale Ale,36926,WI,US,American Pale Ale (APA),Rotating,No notes at this time.,t,1349,...,1248363261,3.0,3.5,3.5,Bock,3.0,3.0,Saaremaa Tume,6.5,51372
358870,187618,RIPTA,34665,RI,US,Belgian Tripel,Rotating,No notes at this time.,f,1349,...,1239199691,4.0,4.0,3.0,Baltic Porter,3.5,3.5,A. Le Coq Jõuluporter,7.5,14411
358871,283124,Rumble Fish,29238,MI,US,American Imperial IPA,Rotating,No notes at this time.,f,1349,...,1233597942,3.5,4.5,3.5,Baltic Porter,3.5,3.5,A. Le Coq Jõuluporter,7.5,14411


In [105]:
df_ny.dropna(inplace=True)

<ipython-input-105-500e1490662f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ny.dropna(inplace=True)


In [106]:
df_ny

,id,name,brewery_id,state,country,style,availability,notes,retired
0,202522,Olde Cogitator,2199,CA,US,English Oatmeal Stout,Rotating,No notes at this time.,f
2,214879,Scottish Right,44306,IN,US,Scottish Ale,Year-round,No notes at this time.,t
3,320009,MegaMeow Imperial Stout,4378,WA,US,American Imperial Stout,Winter,Every time this year,f
4,246438,Peaches-N-Cream,44617,PA,US,American Cream Ale,Rotating,No notes at this time.,f
6,108605,Icon Sender,22598,CA,US,American Lager,Year-round,No notes at this time.,f
...,...,...,...,...,...,...,...,...,...
358865,258269,Brewers Select #019 High Pressure Lager,28281,WA,US,American Lager,Rotating,No notes at this time.,f
358867,146613,American Pale Ale,36926,WI,US,American Pale Ale (APA),Rotating,No notes at this time.,t
358870,187618,RIPTA,34665,RI,US,Belgian Tripel,Rotating,No notes at this time.,f
358871,283124,Rumble Fish,29238,MI,US,American Imperial IPA,Rotating,No notes at this time.,f


In [107]:
df_ny = df_ny[df_ny['notes'] != 'No notes at this time.']
df_ny.set_index('id')

,name,brewery_id,state,country,style,availability,notes,retired
id,,,,,,,,
320009,MegaMeow Imperial Stout,4378,WA,US,American Imperial Stout,Winter,Every time this year,f
303023,The Sky Is High (wet Hopped Pale Ale),46616,NJ,US,American Pale Ale (APA),Limited (brewed once),"This hazy pale ale is brewed with fresh Cascade, Centennial, and Chinook hops from Sky High Hops. It is then dry hopped heavily with Cascade and Chinook.",t
165236,Fortuna,30,CO,US,American Wild Ale,Limited (brewed once),Sour ale that is aged in Tequila barrels with lime zest and salt.,t
200033,Forest Nymph,42273,MN,US,Rye Beer,Limited (brewed once),"Deep amber with tan foam. Fragrance of spicy rye, caramelized raisins and toasted fall spices. Flavors are reminiscent of freshly baked cinnamon raisin bread, light bodied and refreshing.",t
55,Redhook Double Black Stout,18134,NH,US,American Imperial Stout,Rotating,Brewed with coffee.,f
...,...,...,...,...,...,...,...,...
154296,A Page From Camus,28019,CA,US,Belgian Pale Ale,Limited (brewed once),"Tart, blended Belgian blonde.",t
227757,Soleil,44599,NY,US,American Wild Ale,Spring,"Sour Farmhouse Ale conditioned on pineapple, marigold flowers and rose petals",f
149436,Don't Fear The Creeper,37969,CA,US,American Porter,Limited (brewed once),"Our First Collaboration Beer with Lonely Creeper Records. Robust Porter brewed with Cinnamon Toast Crunch. American 2 Row, Chocolate, Victory, Black Patent, Crystal and Roasted Barley Malts, English Hops and California Ale Yeast.",t


In [108]:
new_df = df_ny.join(df_reviews, how='inner')

In [109]:
len(df_ny['name'].unique())

40176

In [110]:
new_df

,id,name,brewery_id,state,country,style,availability,notes,retired,beer_id,username,date,text,look,smell,taste,feel,overall,score
3,320009,MegaMeow Imperial Stout,4378,WA,US,American Imperial Stout,Winter,Every time this year,f,125646,GratefulBeerGuy,2017-12-20,"0% 16 oz can. Funny story: As I finally walked in the doors after a 45 min wait in line and freezing temps the sweet sound of the Grateful Dead's Sugar Magnolia greeted me from the TreeHouse sound system. The bottom of the can reads: ""Going where the wind goes, bloomin' like a red rose"" A white haze to the yellow and golden liquid. Thick and healthy, totally unfiltered. Brawny white foam cap, thick, all-white clumps. Huge lacing left over. The aroma has a very zesty citrus hop effect, mellon and mango, grainy earthiness, tropical fruit blend with a bitter to sweet effect, then a peppery kick at the end. Very aromatic. The flavor is just bursting with complex hops, zesty earthy tones, sweet orange, peppery malt, clean fresh feel and overall vibe. A crispy bite wakes you up, full and lush mouthfeel follows from a totally unfiltered expereince. The feel and flavor finishes with a fun, earthy, zesty dry bite. Tropical juicy, zesty citrus, zippy golden wheat malt, melons, rustic earthiness sums up the taste pretty well. Levels of complexity are deep. It's an interesting ride for sure. Overall, this one stands somewhere near the top of the New England IPA's.",4.75,4.75,4.50,4.50,4.50,4.58
16,303023,The Sky Is High (wet Hopped Pale Ale),46616,NJ,US,American Pale Ale (APA),Limited (brewed once),"This hazy pale ale is brewed with fresh Cascade, Centennial, and Chinook hops from Sky High Hops. It is then dry hopped heavily with Cascade and Chinook.",t,125646,hankf44,2017-12-07,,4.75,4.75,4.75,5.00,4.75,4.78
27,165236,Fortuna,30,CO,US,American Wild Ale,Limited (brewed once),Sour ale that is aged in Tequila barrels with lime zest and salt.,t,205644,SMH_NWI,2017-04-14,,4.00,4.00,3.50,3.50,3.75,3.70
37,200033,Forest Nymph,42273,MN,US,Rye Beer,Limited (brewed once),"Deep amber with tan foam. Fragrance of spicy rye, caramelized raisins and toasted fall spices. Flavors are reminiscent of freshly baked cinnamon raisin bread, light bodied and refreshing.",t,150672,byronic,2016-04-14,Poured a bit lively and had to wait for it to settle very nice lacing. Slightly hazy golden straw. Floral notes maybe a hint of citrus. Quite smooth on the palate a touch bitter and definitely a pungent finish as advertised. Somewhat of a dry finish and something about the lingering taste is off. The mouth coating is a touch creamy.,3.75,3.75,3.75,3.75,3.75,3.75
43,55,Redhook Double Black Stout,18134,NH,US,American Imperial Stout,Rotating,Brewed with coffee.,f,150672,wanderingspirit,2015-12-20,"Quite balanced and distinct IPA of a West Coast style IPA from this small prairie city brewery. Packs a punch at 7abv but the bite is layered by pine and citrus aroma and taste, but a nice malt finish and stands up to many west coast IPAs. Nice start for Black Bridge.",4.25,4.50,4.25,4.50,4.25,4.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358792,304308,Breakfast Beer,36594,CA,US,German Hefeweizen,Limited (brewed once),German-style Mini Weisse with oranges.,t,81777,spoony,2012-09-05,"500 ml bottle into a pint glass. A-The brew had an amber color with honey and apricot tones. Mine poured with virtually no head and very little visible carbonation. S-The smell was dominated by a phenolic, chemical note. It was quite sweet and a little off-putting. I didn't get any fruit, floral, hop, or yeast notes. T-Pretty bland. There was a touch of malt and a hint of apple and floral flavor, but that off-putting chemical quality persists. M-slightly oily. No carbonation meant that it drank easily, but without much in the way of taste, I took my time finishing my one glass. O-This came recommended by two guys at the liquor store in Vancouver and I was intrigued by the idea of drinking a beer brew

In [111]:
fsm = DummyRegressor()

In [112]:
len(new_df['style'].unique())

109

In [113]:
new_df['style'].unique()

# Clearly we need to set target variables for all of these different types of beers

array(['American Imperial Stout', 'American Pale Ale (APA)',
       'American Wild Ale', 'Rye Beer', 'American IPA',
       'Foreign / Export Stout', 'Belgian Strong Pale Ale',
       'Leipzig Gose', 'Winter Warmer', 'Baltic Porter',
       'American Adjunct Lager', 'American Porter',
       'American Imperial IPA', 'American Pale Wheat Ale',
       'Belgian Strong Dark Ale', 'German Kölsch', 'English Old Ale',
       'American Blonde Ale', 'Belgian Tripel', 'Belgian Saison',
       'Fruit and Field Beer', 'German Hefeweizen', 'German Rauchbier',
       'Bohemian Pilsener', 'English Oatmeal Stout', 'American Lager',
       'American Brown Ale', 'American Black Ale', 'Scottish Ale',
       'German Bock', 'German Märzen / Oktoberfest', 'German Doppelbock',
       'American Cream Ale', 'Belgian Blonde Ale', 'Belgian Witbier',
       'American Stout', 'American Amber / Red Lager',
       'American Amber / Red Ale', 'German Pilsner',
       'English Sweet / Milk Stout', 'English Dark Mild A

In [114]:
le = LabelEncoder()
le.fit(new_df['style'])
le_style = le.transform(new_df['style'])
len(le_style)

23037

In [115]:
new_df['le_style'] = le_style

In [116]:
new_df[new_df['name'] == 'Jubilee']

,id,name,brewery_id,state,country,style,availability,notes,retired,beer_id,username,date,text,look,smell,taste,feel,overall,score,le_style
273,123681,Jubilee,14952,VA,US,Belgian Strong Dark Ale,Limited (brewed once),Aged in premium Bowman's bourbon barrels for over a year.,t,125646,Bammer21,2017-11-16,,4.25,4.75,4.5,4.25,4.5,4.52,38
152333,248142,Jubilee,345,PA,US,Bière de Champagne / Bière Brut,Rotating,HOPS: German Blanc,f,57048,Ricochet,2010-05-07,"This is a Genesee product? Really? I was never a big fan of JWD Honey Brown, but I was an early positive reviewer when they came out with a decent pale ale and amber. Then they faltered a bit with a few lackluster tries. Now just ""Dundee"" (except the original honey brown), they have really stepped up and the past few releases have been very good, but GREAT for style vs. price. They are bargains just like the pale ale was years ago. When JWD released a porter, I thought it was weak and barely resembling the style. This stout on the other hand... Poured black with a quickly fading head and a mild sticky lace but it beaded when poured. Great sign. Smelled like a sweet stout. All malt, slight coffee, a little maple syrup. Mouth was, well, as expected. Thin, slight foam, bread like outside, slight bitter and sweet coffee going down. Sweet from malt to start, slight bitter. Touch of coffee but more maple syrup. Alcohol tempered and just a bit of the bread flavor. I feel that maybe the beer misses a bit on follow through as there was a point where I wished it had more flavor, but this is still a really good beverage and top of the line for Dundee. I'd get it again.",4.50,4.00,4.0,4.00,3.5,3.93,43


Let's try creating a content based system; picking beers that have similar ratings to what you picked.

In [117]:
rcmd = new_df[['beer_id', 'style', 'text', 'notes',
               'name', 'look', 'smell', 'taste', 'feel', 'overall', 'score']]

In [118]:
len(rcmd['name'].unique())

22155

In [119]:
rcmd

,beer_id,style,text,notes,name,look,smell,taste,feel,overall,score
3,125646,American Imperial Stout,"0% 16 oz can. Funny story: As I finally walked in the doors after a 45 min wait in line and freezing temps the sweet sound of the Grateful Dead's Sugar Magnolia greeted me from the TreeHouse sound system. The bottom of the can reads: ""Going where the wind goes, bloomin' like a red rose"" A white haze to the yellow and golden liquid. Thick and healthy, totally unfiltered. Brawny white foam cap, thick, all-white clumps. Huge lacing left over. The aroma has a very zesty citrus hop effect, mellon and mango, grainy earthiness, tropical fruit blend with a bitter to sweet effect, then a peppery kick at the end. Very aromatic. The flavor is just bursting with complex hops, zesty earthy tones, sweet orange, peppery malt, clean fresh feel and overall vibe. A crispy bite wakes you up, full and lush mouthfeel follows from a totally unfiltered expereince. The feel and flavor finishes with a fun, earthy, zesty dry bite. Tropical juicy, zesty citrus, zippy golden wheat malt, melons, rustic earthiness sums up the taste pretty well. Levels of complexity are deep. It's an interesting ride for sure. Overall, this one stands somewhere near the top of the New England IPA's.",Every time this year,MegaMeow Imperial Stout,4.75,4.75,4.50,4.50,4.50,4.58
16,125646,American Pale Ale (APA),,"This hazy pale ale is brewed with fresh Cascade, Centennial, and Chinook hops from Sky High Hops. It is then dry hopped heavily with Cascade and Chinook.",The Sky Is High (wet Hopped Pale Ale),4.75,4.75,4.75,5.00,4.75,4.78
27,205644,American Wild Ale,,Sour ale that is aged in Tequila barrels with lime zest and salt.,Fortuna,4.00,4.00,3.50,3.50,3.75,3.70
37,150672,Rye Beer,Poured a bit lively and had to wait for it to settle very nice lacing. Slightly hazy golden straw. Floral notes maybe a hint of citrus. Quite smooth on the palate a touch bitter and definitely a pungent finish as advertised. Somewhat of a dry finish and something about the lingering taste is off. The mouth coating is a touch creamy.,"Deep amber with tan foam. Fragrance of spicy rye, caramelized raisins and toasted fall spices. Flavors are reminiscent of freshly baked cinnamon raisin bread, light bodied and refreshing.",Forest Nymph,3.75,3.75,3.75,3.75,3.75,3.75
43,150672,American Imperial Stout,"Quite balanced and distinct IPA of a West Coast style IPA from this small prairie city brewery. Packs a punch at 7abv but the bite is layered by pine and citrus aroma and taste, but a nice malt finish and stands up to many west coast IPAs. Nice start for Black Bridge.",Brewed with coffee.,Redhook Double Black Stout,4.25,4.50,4.25,4.50,4.25,4.34
...,...,...,...,...,...,...,...,...,...,...,...
358792,81777,German Hefeweizen,"500 ml bottle into a pint glass. A-The brew had an amber color with honey and apricot tones. Mine poured with virtually no head and very little visible carbonation. S-The smell was dominated by a phenolic, chemical note. It was quite sweet and a little off-putting. I didn't get any fruit, floral, hop, or yeast notes. T-Pretty bland. There was a touch of malt and a hint of apple and floral flavor, but that off-putting chemical quality persists. M-slightly oily. No carbonation meant that it drank easily, but without much in the way of taste, I took my time finishing my one glass. O-This came recommended by two guys at the liquor store in Vancouver and I was intrigued by the idea of drinking a beer brewed in Greenland, so perhaps I had unreasonably high hopes, but I found it disappointing all around. Perhaps I got a bad/old bottle.",German-style Mini Weisse with oranges.,Breakfast Beer,3.00,2.50,3.00,3.00,2.50,2.78
358799,81883,American Imperial IPA,,"Brewed with Chinook, Amarillo, Centennial, Mosaic, and Eureka.",Hop Nookie,3.50,4.00,3.50,3.50,3.75,3.67
358818,94229,Belgian Pale Ale,"On tap at Harrah's Casino in Philadelphia. What am I doing drinking this at a casino? Why is it in a pint glas

In [120]:
# rcmd.set_index(['beer_id'], inplace=True)
rcmd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23037 entries, 3 to 358846
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   beer_id  23037 non-null  int64  
 1   style    23037 non-null  object 
 2   text     23037 non-null  object 
 3   notes    23037 non-null  object 
 4   name     23037 non-null  object 
 5   look     23037 non-null  float64
 6   smell    23037 non-null  float64
 7   taste    23037 non-null  float64
 8   feel     23037 non-null  float64
 9   overall  23037 non-null  float64
 10  score    23037 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 2.1+ MB


In [121]:
# Looks like we have about 1k rows where there are reviews for the same beers.
# Let's see how we can deal with those.
# Maybe can use Grouby?

In [122]:
rec_group = rcmd.groupby(['name'], as_index=False).mean()
rec_group['beer_id'] = rec_group['beer_id'].astype(str).apply(lambda x: x.replace('.0',''))

rec_group['beer_id'] = rec_group['beer_id'].astype(float)
rec_group.set_index(['beer_id'], inplace=True)
rec_group.info()


<class 'pandas.core.frame.DataFrame'>
Float64Index: 22155 entries, 76816.0 to 50697.0
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     22155 non-null  object 
 1   look     22155 non-null  float64
 2   smell    22155 non-null  float64
 3   taste    22155 non-null  float64
 4   feel     22155 non-null  float64
 5   overall  22155 non-null  float64
 6   score    22155 non-null  float64
dtypes: float64(6), object(1)
memory usage: 1.4+ MB


In [123]:
len(rec_group)

22155

In [124]:
len(rcmd)

23037

In [125]:
rec_group = pd.merge(rcmd, rec_group, how='right')

In [126]:
rec_group.dropna(inplace=True)

In [127]:
rec_idens = rec_group[['beer_id', 'style', 'name', 'text', 'notes']]
rec_idens['combined_text'] = rec_idens['style'] + ' ' + rec_idens['text'] + ' ' + rec_idens['notes']
rec_idens.drop(columns=['style', 'text', 'notes', 'beer_id'], inplace=True)
rec_idens = rec_idens.applymap(lambda x: x.lower())
rec_idens = rec_idens.applymap(lambda x: x.translate(x.maketrans('', '', string.punctuation)))
sw = stopwords.words('english')
sw
rec_idens['combined_text'] = rec_idens['combined_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)]))

rec_idens = rec_idens.applymap(lambda x: x.translate(x.maketrans('', '', '0123456789')))
rec_idens

<ipython-input-127-390bdae73fcb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rec_idens['combined_text'] = rec_idens['style'] + ' ' + rec_idens['text'] + ' ' + rec_idens['notes']
/Users/adamlarsen/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,name,combined_text
0,creamsickle gold ale cask,herb spice beer gold ale aged vanilla bean orange peel citra amarillo
1,inc,american stout bomber pint glass slightly cloudy murky sediment mixing upon pour intended though unavoidable look ogranic honey crisp apple juice head reasonable fingers lacing though serious sticks around duration pineapple tropical fruit close eyes feel sun back neck sand toes hints sweetened coconut milk intensify warms utterly fantastic expecting fresh lavish hops burst like grenade mouth cant begin tell inviting citrus actually seems ive got umbrella drink sitting motorboat islands slightly acidic though fairly creamy thick carbonation ever slightly sharp offputting alcohol nowhere found dugana overly cloying like lot dipas ive aftertaste sticks around certainly wont smell like brewery night dugana fabulous beer delicious bomber must say tap genuine experience one top ten beers ive last year guarantee youll thank recommending see tap handle look enjoy rich roasty aromas bakers chocolate give way soft rounded mouthfeel flavors medium roast coffee burnt caramel firm restrained bitterness layers roast additional depth flavor wonderfully balanced flavors perfectly incorporated
2,lager,vienna lager beer decent overall value get pay pretty dark red copper hue stien nice palet thick creamy feel fairly low carbination intense aromas alcohol ripe fruit without sweet sticky nice carmely flavor somewhat malty savory beer nice convorsational sipper would gladly purchase put slightly american twist classic vienna lager make we’re calling “every day beer” beer hints toasted bran touch malt sweetness balanced slightly heavy handed use delicate noble hops finish crisp keep coming back hope say “gimme lager”
3,mike saw a sasquatch session ale,american blonde ale true summer ale central oregon golden ale light smooth taste craftsmanship easy drinking ale smooth blend cascade sterling hops finished touch honey malt balance great introduction new world craft brewing established craft enthusiasts barrel summer ale good reprieve overly hopped overly malty alternatives
4,o gold,american light lager lightest beer great attributes light american lager unmatched freshness taste local brewpub offer like greattasting light beer without calories
...,...,...
22152,‘home’ brew national homebrewers conference bottle,american ipa huge thanks sideshowe sharing rare gem pours hazy dark orange color one finger offwhite head slowly settles lasting ring spotty lacing left behind earthy musty aroma lots barnyardhorsey character hints oak funky bugs lactic acid cobweb followed faint fruit acetic vinegar expecting nose bursting cherry pretty subtle smells pretty good nice funk kind one dimensional medium bodied tart lactic funk isnt acetic harsh decently sour much funky anything else slight acetic quality touch vinegar finish solent like sweet cherries slightly integrated subtle vinous character fruit skin cherry pit finishes dry lingering acidity brewmaster paul segura team karl strauss call commercial brewery “home” folks working brewery trace roots back homebrew system inspired karl strauss ‘home’ brew san diegostyle india pale ale simple malt bill paired array flavorful aromatic hops makes ipa pays homage dedication determination ingenuity homebrewers
22153,‘ssippi squeeze,american ipa golden hazy dryhopped new england ipa packed citra mosaic hops
22154,“chewy” double ipa,american imperial ipa double ipa brewed north american cultivated hops malted barley special attention two hop varieties utilized brew dryhop millenium falconer’s flight dipa soars abv ibu bitterness hit speed light
22155,分 桃 fēn táo,american wild ale oz bottle date says abv wow pours ambertinted golden moderate white head sticky lacing incredibly balanced aromas nectarine peach tangerine little orange ripe grapefruit warm tropicals tad lemon pine tad dank mostly sweet taste hop forward bitter incredibly balanced pine resin grapefruit orange tropical fruit stone fruit 

In [128]:
rec_group['combined_text'] = rec_idens['combined_text']

In [129]:
rec_group.reset_index(inplace=True)
rec_group.drop(columns=['index'], inplace=True)
df_streamlit = rec_group.to_csv('../../df_streamlit.csv',index=False)
test = rec_group

In [130]:
# rec_idens = rec_group[['beer_id', 'style', 'name', 'text', 'notes']]
# rec_idens['combined_text'] = rec_idens['style'] + ' ' + rec_idens['text'] + ' ' + rec_idens['notes']
# rec_idens.drop(columns=['style', 'text', 'notes'], inplace=True)
# rec_idens

In [131]:
rec_numbs = rec_group[['look', 'smell', 'taste', 'feel', 'overall', 'score']]
rec_numbs_array = np.array(rec_numbs)
len(rec_numbs)
len(rec_numbs_array)

21722

In [132]:
# rec_group['score_combined'] = ((rec_group['look'] + rec_group['smell'] + rec_group['taste']
# + rec_group['feel'] + rec_group['overall'] + rec_group['score']) / 6).round(3)

# rec_group


In [133]:
rec_group.isna().sum()
rec_group

,beer_id,style,text,notes,name,look,smell,taste,feel,overall,score,combined_text
0,76816.0,Herb and Spice Beer,,"Gold Ale aged on vanilla bean, orange peel, citra + Amarillo","""Creamsickle"" Gold Ale Cask",3.75,3.75,4.00,4.00,4.00,3.93,herb spice beer gold ale aged vanilla bean orange peel citra amarillo
1,53167.0,American Stout,"From bomber to pint glass A: Slightly cloudy and murky, with sediment mixing upon pour (not intended though unavoidable). It has the look of ogranic Honey Crisp apple juice. The head is reasonable, 1.75 fingers. The lacing though is serious, and sticks around for the duration. S: Pineapple and tropical fruit. If I close my eyes I can feel the sun on the back of my neck and the sand under my toes. Hints of sweetened coconut milk intensify as it warms, which is utterly fantastic. T: Just what I was expecting - fresh, lavish hops that burst like a grenade in my mouth. I can't begin to tell you how inviting the citrus is. It actually seems as if I've got an umbrella in my drink while sitting on a motorboat in the islands. M: Slightly acidic, though fairly creamy and thick. The carbonation is ever so slightly sharp, but it's not off-putting. The alcohol is nowhere to be found. O: DuganA is not overly cloying like a lot of DIPA's I've had. The aftertaste sticks around, but you certainly won't smell like a brewery all night. DuganA is a fabulous beer. For as delicious as this bomber was, I must say that having it ON TAP is a genuine experience. It is one of the top ten beers I've had in the last year, and I can guarantee you'll thank me for recommending it. If you see the tap handle, look no further and enjoy.",Rich and roasty aromas of bakers chocolate give way to a soft and rounded mouthfeel and flavors of medium roast coffee and burnt caramel. A firm but restrained bitterness layers with the roast for additional depth of flavor. Wonderfully balanced with all of the flavors perfectly... Incorporated.,"""INC.""",3.50,4.50,4.50,4.00,4.50,4.39,american stout bomber pint glass slightly cloudy murky sediment mixing upon pour intended though unavoidable look ogranic honey crisp apple juice head reasonable fingers lacing though serious sticks around duration pineapple tropical fruit close eyes feel sun back neck sand toes hints sweetened coconut milk intensify warms utterly fantastic expecting fresh lavish hops burst like grenade mouth cant begin tell inviting citrus actually seems ive got umbrella drink sitting motorboat islands slightly acidic though fairly creamy thick carbonation ever slightly sharp offputting alcohol nowhere found dugana overly cloying like lot dipas ive aftertaste sticks around certainly wont smell like brewery night dugana fabulous beer delicious bomber must say tap genuine experience one top ten beers ive last year guarantee youll thank recommending see tap handle look enjoy rich roasty aromas bakers chocolate give way soft rounded mouthfeel flavors medium roast coffee burnt caramel firm restrained bitterness layers roast additional depth flavor wonderfully balanced flavors perfectly incorporated
2,52283.0,Vienna Lager,"This beer is a decent overall value being that you get what you pay for. It has a pretty, dark red copper hue in the stien. Nice on the palet with a thick creamy feel. Fairly low on carbination. Intense aromas of alcohol and ripe fruit without being sweet and sticky. Nice carmely flavor, somewhat malty and savory. This beer is a nice convorsational sipper that I would gladly purchase again.","We put a slightly American twist on a classic Vienna Lager to make what we’re calling an “Every Day Beer.” The beer has hints of toasted bran, with a touch of malt sweetness balanced by a slightly heavy handed use of delicate noble hops to finish crisp and keep you coming back for more. We hope you just say, “Gimme a Lager.”","""Lager""",3.50,4.00,4.00,3.50,4.00,3.92,vienna lager beer decent overall value get pay pretty dark red copper hue stien nice palet thick creamy feel fairly lo

In [134]:
rec_group_2 = rec_group.drop(['name', 'style', 'beer_id', 'text', 'notes', 'combined_text'], axis=1)
rec_group_2 = np.array(rec_group_2)
rec_group_2
len(rec_group_2)

21722

In [135]:
numerators = np.array([rec_group_2[0].dot(beers) for beers in rec_group_2[0:]])
denominators = np.array([np.sqrt(sum(rec_group_2[0]**2)) *\
                         np.sqrt(sum(beers**2)) for beers in rec_group_2[0:]])




results = pd.DataFrame((numerators / denominators), columns=['recommendation'])
rec_group.drop([0], inplace=True)
rec_group = rec_group.join(results)


In [136]:
rec_group.isna().sum()

beer_id           0
style             0
text              0
notes             0
name              0
look              0
smell             0
taste             0
feel              0
overall           0
score             0
combined_text     0
recommendation    0
dtype: int64

In [137]:
rec_group.tail()

,beer_id,style,text,notes,name,look,smell,taste,feel,overall,score,combined_text,recommendation
21717,58838.0,American IPA,"Huge thanks to sideshowe for sharing this rare gem! Pours a hazy dark orange color with a one finger off-white head that slowly settles into a lasting ring. Spotty lacing left behind. Earthy, musty aroma with lots of barnyard/horsey character. Hints of oak, funky bugs, lactic acid and cobweb followed by faint fruit and acetic vinegar. I was expecting the nose to be bursting with cherry but it's pretty subtle. Smells pretty good with some nice funk but kind of one dimensional. Medium bodied with a tart, lactic funk that isn't acetic or harsh. Decently sour but not too much do. More funky than anything else with a slight acetic quality. Just a touch of vinegar in the finish but not solent like. Not very sweet with the cherries again being slightly integrated. Subtle vinous character with some fruit skin and cherry pit. Finishes dry with some lingering acidity.","Brewmaster Paul Segura and his team at Karl Strauss call their commercial brewery “home.” Most of the folks working at the brewery can trace their roots back to a homebrew system, which is what inspired Karl Strauss ‘Home’ Brew, a San Diego-style India pale ale. The simple malt bill paired with an array of flavorful and aromatic hops makes for an IPA that pays homage to the dedication, determination and ingenuity of homebrewers.",‘Home’ Brew (2015 National Homebrewers Conference Bottle),3.5,4.00,4.00,4.00,4.00,3.97,american ipa huge thanks sideshowe sharing rare gem pours hazy dark orange color one finger offwhite head slowly settles lasting ring spotty lacing left behind earthy musty aroma lots barnyardhorsey character hints oak funky bugs lactic acid cobweb followed faint fruit acetic vinegar expecting nose bursting cherry pretty subtle smells pretty good nice funk kind one dimensional medium bodied tart lactic funk isnt acetic harsh decently sour much funky anything else slight acetic quality touch vinegar finish solent like sweet cherries slightly integrated subtle vinous character fruit skin cherry pit finishes dry lingering acidity brewmaster paul segura team karl strauss call commercial brewery “home” folks working brewery trace roots back homebrew system inspired karl strauss ‘home’ brew san diegostyle india pale ale simple malt bill paired array flavorful aromatic hops makes ipa pays homage dedication determination ingenuity homebrewers,0.999313
21718,65038.0,American IPA,,A golden & hazy dry-hopped New England IPA packed with Citra & Mosaic hops.,‘Ssippi Squeeze,3.0,2.75,2.75,2.75,2.75,2.77,american ipa golden hazy dryhopped new england ipa packed citra mosaic hops,0.998462
21719,50697.0,American Imperial IPA,,This Double IPA is brewed with North American cultivated hops & malted barley. Special attention to the two hop varieties utilized in the brew & dry-hop: Millenium & Falconer’s Flight. This DIPA soars in at 6.9% abv & 100 IBU with a bitterness that will hit you at the speed of light!,“Chewy” Double IPA,4.5,4.00,4.50,4.00,4.25,4.28,american imperial ipa double ipa brewed north american cultivated hops malted barley special attention two hop varieties utilized brew dryhop millenium falconer’s flight dipa soars abv ibu bitterness hit speed light,0.998442
21720,74986.0,American Wild Ale,"12 oz bottle with no date, says 8% ABV (not 7.85%). Wow! Pours amber-tinted golden, moderate white head with sticky lacing. Incredibly balanced aromas of nectarine and peach, tangerine and a little orange and ripe grapefruit, warm tropicals, just a tad lemon, some pine, a tad dank, mostly sweet. Taste is hop forward and bitter but incredibly balanced, with pine resin and grapefruit, orange, tropical fruit, some stone fruit, with a sweetness that cuts through the resinous hops. All the components line up humbly in their place, waiting for you to explore them, never overstepping, and creating a symphony on the palate. Feel is medium bodied, creamy enough. 

Let's try creating a content based system; picking beers that have similar ratings to what you picked.

In [138]:
rec_group['recommendation'].max()

1.0

In [139]:
has = rec_group[rec_group['style'] == "Herb and Spice Beer"]
has.sort_values(['recommendation'], ascending=False)

,beer_id,style,text,notes,name,look,smell,taste,feel,overall,score,combined_text,recommendation
6695,64158.0,Herb and Spice Beer,"Ample haze runs through a mostly golden beer. Interesting aroma here, mango-kiwi like fruit. Clean sour, brett notes not noticeable. Drinks like an upland kiwi lambic but without the massive sourness or harsh acidity. Lacto sour. Probably my favorite Jackie O's beer, very well done, cool unique spin on a relatively unknown or underutilized fruit.","A craft beer spin on a German bar classic, EuroDiesel is an American wheat ale brewed with traditional cola flavors for a delicious drinking experience that will have you cruising down the road to refreshment. PROST!",Eurodiesel Traveler Cola Shandy,4.00,4.00,4.25,4.25,4.25,4.18,herb spice beer ample haze runs mostly golden beer interesting aroma mangokiwi like fruit clean sour brett notes noticeable drinks like upland kiwi lambic without massive sourness harsh acidity lacto sour probably favorite jackie os beer well done cool unique spin relatively unknown underutilized fruit craft beer spin german bar classic eurodiesel american wheat ale brewed traditional cola flavors delicious drinking experience cruising road refreshment prost,0.999999
15722,78386.0,Herb and Spice Beer,"22oz red waxed bottle poured into a CBS oversized snifter. A - Pours a slightly hazy mahogany tinted amber with a short half finger of foam the falls quickly and leaves behind a thick, foamy collar around the edge of the glass. Traces of lacing linger on the glass. S - Interesting aroma - a bit different. First whiff, I get some nice fresh grassy hops - a pleasant opening surprise that I wasn't expecting. Behind that I get some potent bourbon and a strong helping of sweet, caramelly, toffee malt. Smells very well balanced - approaching, but not quite barley wine like in impression. T - I continue to get grassy hops in the flavor too which is very welcome and well placed here. Sweet caramel malt, traces of earthy malt, lightly fruity/estery. Sweet bourbon notes too - integrated nicely without an overbearing intensity. First and foremost this is still a red ale - the barrel adds bourbon nuances with a touch of booziness, but well controlled. M - Quite full bodied, thick and rather syrupy in feel. Lower level carbonation fits in well adding only a low tingle on the tongue. More tingle from the bourbon boozy warmth, but again at a low level. Just the way I like it - enough to be well noticed, but not so much as to overpower the base beer. O - Had no idea what to expect going into this - no experience with the beer or the brewery itself...not to mention bourbon barrel imperial reds aren't exactly mainstream either. I find it rather enjoyable. Stops short of being an out and out barley wine in character. Feels lighter although the fullness of the body cannot be denied. I like the firm grassy hop character in the nose and the taste - really brightens things up in both areas. Balance is quite good, drinkability is far better than I expected. I would definitely have this one again.",A winter seasonal brew infused with fresh sprigs of the red spruce tree and a dash of cinnamon that breaks from traditionally heavy holiday brews with a light body and refreshing zest.,Red Spruce Bitter,4.00,4.00,4.25,4.50,4.25,4.20,herb spice beer oz red waxed bottle poured cbs oversized snifter pours slightly hazy mahogany tinted amber short half finger foam falls quickly leaves behind thick foamy collar around edge glass traces lacing linger glass interesting aroma bit different first whiff get nice fresh grassy hops pleasant opening surprise wasnt expecting behind get potent bourbon strong helping sweet caramelly toffee malt smells well balanced approaching quite barley wine like impression continue get grassy hops flavor welcome well placed sweet caramel malt traces earthy malt lightly fruityestery sweet bourbon notes integrated nicely without overbearing intensity first foremost still red ale barrel adds bou

A decent model right now, just need to find out how to implement a variable by choosing a beer name.

I also would love to be able to use count of words in descriptions in order to give a better idea of what descriptions match each other. Let's see if we can use count vectorizer in order to give us a matrix of words.

In order to do this, we'll use rec_idens

In [140]:
# rec_idens.drop(columns=['beer_id', 'name'], inplace=True)

In [141]:
# rec_idens = rec_idens.applymap(lambda x: x.lower())
# rec_idens

In [142]:
# rec_idens = rec_idens.applymap(lambda x: x.translate(x.maketrans('', '', string.punctuation)))
# rec_idens


In [143]:
sw = stopwords.words('english')
sw

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [144]:
sw = stopwords.words('english')
sw
rec_idens['combined_text'] = rec_idens['combined_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)]))

rec_idens = rec_idens.applymap(lambda x: x.translate(x.maketrans('', '', '0123456789')))

In [145]:
rec_idens

,name,combined_text
0,creamsickle gold ale cask,herb spice beer gold ale aged vanilla bean orange peel citra amarillo
1,inc,american stout bomber pint glass slightly cloudy murky sediment mixing upon pour intended though unavoidable look ogranic honey crisp apple juice head reasonable fingers lacing though serious sticks around duration pineapple tropical fruit close eyes feel sun back neck sand toes hints sweetened coconut milk intensify warms utterly fantastic expecting fresh lavish hops burst like grenade mouth cant begin tell inviting citrus actually seems ive got umbrella drink sitting motorboat islands slightly acidic though fairly creamy thick carbonation ever slightly sharp offputting alcohol nowhere found dugana overly cloying like lot dipas ive aftertaste sticks around certainly wont smell like brewery night dugana fabulous beer delicious bomber must say tap genuine experience one top ten beers ive last year guarantee youll thank recommending see tap handle look enjoy rich roasty aromas bakers chocolate give way soft rounded mouthfeel flavors medium roast coffee burnt caramel firm restrained bitterness layers roast additional depth flavor wonderfully balanced flavors perfectly incorporated
2,lager,vienna lager beer decent overall value get pay pretty dark red copper hue stien nice palet thick creamy feel fairly low carbination intense aromas alcohol ripe fruit without sweet sticky nice carmely flavor somewhat malty savory beer nice convorsational sipper would gladly purchase put slightly american twist classic vienna lager make we’re calling “every day beer” beer hints toasted bran touch malt sweetness balanced slightly heavy handed use delicate noble hops finish crisp keep coming back hope say “gimme lager”
3,mike saw a sasquatch session ale,american blonde ale true summer ale central oregon golden ale light smooth taste craftsmanship easy drinking ale smooth blend cascade sterling hops finished touch honey malt balance great introduction new world craft brewing established craft enthusiasts barrel summer ale good reprieve overly hopped overly malty alternatives
4,o gold,american light lager lightest beer great attributes light american lager unmatched freshness taste local brewpub offer like greattasting light beer without calories
...,...,...
22152,‘home’ brew national homebrewers conference bottle,american ipa huge thanks sideshowe sharing rare gem pours hazy dark orange color one finger offwhite head slowly settles lasting ring spotty lacing left behind earthy musty aroma lots barnyardhorsey character hints oak funky bugs lactic acid cobweb followed faint fruit acetic vinegar expecting nose bursting cherry pretty subtle smells pretty good nice funk kind one dimensional medium bodied tart lactic funk isnt acetic harsh decently sour much funky anything else slight acetic quality touch vinegar finish solent like sweet cherries slightly integrated subtle vinous character fruit skin cherry pit finishes dry lingering acidity brewmaster paul segura team karl strauss call commercial brewery “home” folks working brewery trace roots back homebrew system inspired karl strauss ‘home’ brew san diegostyle india pale ale simple malt bill paired array flavorful aromatic hops makes ipa pays homage dedication determination ingenuity homebrewers
22153,‘ssippi squeeze,american ipa golden hazy dryhopped new england ipa packed citra mosaic hops
22154,“chewy” double ipa,american imperial ipa double ipa brewed north american cultivated hops malted barley special attention two hop varieties utilized brew dryhop millenium falconer’s flight dipa soars abv ibu bitterness hit speed light
22155,分 桃 fēn táo,american wild ale oz bottle date says abv wow pours ambertinted golden moderate white head sticky lacing incredibly balanced aromas nectarine peach tangerine little orange ripe grapefruit warm tropicals tad lemon pine tad dank mostly sweet taste hop forward bitter incredibly balanced pine resin grapefruit orange tropical fruit stone fruit 

In [146]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df = 1, max_features=200)

In [147]:
tfid_vec = tf.fit_transform(rec_idens['combined_text'])

In [148]:
tfid_vec

<21722x200 sparse matrix of type '<class 'numpy.float64'>'
	with 456505 stored elements in Compressed Sparse Row format>

In [149]:
tf.get_feature_names()

['abv',
 'aged',
 'alcohol',
 'ale',
 'ale apa',
 'almost',
 'also',
 'amber',
 'american',
 'american imperial',
 'american imperial ipa',
 'american imperial stout',
 'american ipa',
 'american pale',
 'american pale ale',
 'american wild',
 'american wild ale',
 'amount',
 'apa',
 'appearance',
 'aroma',
 'aromas',
 'around',
 'back',
 'balance',
 'balanced',
 'barrel',
 'barrels',
 'beer',
 'beers',
 'belgian',
 'belgian saison',
 'best',
 'big',
 'bit',
 'bitter',
 'bitterness',
 'black',
 'blend',
 'bodied',
 'body',
 'bottle',
 'bourbon',
 'brew',
 'brewed',
 'brewing',
 'bright',
 'brown',
 'caramel',
 'carbonation',
 'character',
 'chocolate',
 'citra',
 'citrus',
 'clean',
 'clear',
 'coffee',
 'collaboration',
 'color',
 'complex',
 'could',
 'creamy',
 'crisp',
 'dark',
 'decent',
 'deep',
 'definitely',
 'double',
 'drink',
 'drinkable',
 'dry',
 'earthy',
 'easy',
 'end',
 'english',
 'even',
 'feel',
 'fermented',
 'finger',
 'finish',
 'first',
 'flavor',
 'flavors',
 '

In [150]:
len(tf.get_feature_names())

200

In [151]:
closeness = cosine_similarity(tfid_vec, tfid_vec[0])

In [152]:
closeness

array([[1.        ],
       [0.01414626],
       [0.06564748],
       ...,
       [0.        ],
       [0.14590024],
       [0.05650837]])

In [153]:
closeness.shape

(21722, 1)

In [154]:
rec_idens.shape

(21722, 2)

In [155]:
closeness = pd.DataFrame(closeness)
closeness

,0
0,1.000000
1,0.014146
2,0.065647
3,0.118598
4,0.063537
...,...
21717,0.066336
21718,0.167710
21719,0.000000
21720,0.145900


In [156]:
closeness[1:].idxmax()

0    7515
dtype: int64

In [157]:
rec_idens.iloc[7515]

name             frau perchta                                                       
combined_text    herb spice beer special version weizenbock dosed orange peel nutmeg
Name: 7694, dtype: object

In [158]:
len(rec_idens)

21722

In [159]:
rec_group.iloc[7514]

beer_id           221526                                                                
style             Herb and Spice Beer                                                   
text                                                                                    
notes             A special version of our Weizenbock dosed with Orange Peel and Nutmeg.
name              Frau Perchta                                                          
look              4                                                                     
smell             4.25                                                                  
taste             4                                                                     
feel              4.25                                                                  
overall           4                                                                     
score             4.09                                                                  
combined_text     her

In [160]:
len(rec_group)

21721

In [161]:
# Working on trying to get streamlit to work

In [162]:
bb = test[test['name'] == '"Lager"']

bb['notes']

2    We put a slightly American twist on a classic Vienna Lager to make what we’re calling an “Every Day Beer.” The beer has hints of toasted bran, with a touch of malt sweetness balanced by a slightly heavy handed use of delicate noble hops to finish crisp and keep you coming back for more. We hope you just say, “Gimme a Lager.”
Name: notes, dtype: object

In [164]:
a = 'Bud Light'
aa = test[test['name'] == a]
# aa
print(aa.index)

Int64Index([3498], dtype='int64')


In [165]:
test.drop(aa.index)

,beer_id,style,text,notes,name,look,smell,taste,feel,overall,score,combined_text
1,53167.0,American Stout,"From bomber to pint glass A: Slightly cloudy and murky, with sediment mixing upon pour (not intended though unavoidable). It has the look of ogranic Honey Crisp apple juice. The head is reasonable, 1.75 fingers. The lacing though is serious, and sticks around for the duration. S: Pineapple and tropical fruit. If I close my eyes I can feel the sun on the back of my neck and the sand under my toes. Hints of sweetened coconut milk intensify as it warms, which is utterly fantastic. T: Just what I was expecting - fresh, lavish hops that burst like a grenade in my mouth. I can't begin to tell you how inviting the citrus is. It actually seems as if I've got an umbrella in my drink while sitting on a motorboat in the islands. M: Slightly acidic, though fairly creamy and thick. The carbonation is ever so slightly sharp, but it's not off-putting. The alcohol is nowhere to be found. O: DuganA is not overly cloying like a lot of DIPA's I've had. The aftertaste sticks around, but you certainly won't smell like a brewery all night. DuganA is a fabulous beer. For as delicious as this bomber was, I must say that having it ON TAP is a genuine experience. It is one of the top ten beers I've had in the last year, and I can guarantee you'll thank me for recommending it. If you see the tap handle, look no further and enjoy.",Rich and roasty aromas of bakers chocolate give way to a soft and rounded mouthfeel and flavors of medium roast coffee and burnt caramel. A firm but restrained bitterness layers with the roast for additional depth of flavor. Wonderfully balanced with all of the flavors perfectly... Incorporated.,"""INC.""",3.50,4.50,4.50,4.00,4.50,4.39,american stout bomber pint glass slightly cloudy murky sediment mixing upon pour intended though unavoidable look ogranic honey crisp apple juice head reasonable fingers lacing though serious sticks around duration pineapple tropical fruit close eyes feel sun back neck sand toes hints sweetened coconut milk intensify warms utterly fantastic expecting fresh lavish hops burst like grenade mouth cant begin tell inviting citrus actually seems ive got umbrella drink sitting motorboat islands slightly acidic though fairly creamy thick carbonation ever slightly sharp offputting alcohol nowhere found dugana overly cloying like lot dipas ive aftertaste sticks around certainly wont smell like brewery night dugana fabulous beer delicious bomber must say tap genuine experience one top ten beers ive last year guarantee youll thank recommending see tap handle look enjoy rich roasty aromas bakers chocolate give way soft rounded mouthfeel flavors medium roast coffee burnt caramel firm restrained bitterness layers roast additional depth flavor wonderfully balanced flavors perfectly incorporated
2,52283.0,Vienna Lager,"This beer is a decent overall value being that you get what you pay for. It has a pretty, dark red copper hue in the stien. Nice on the palet with a thick creamy feel. Fairly low on carbination. Intense aromas of alcohol and ripe fruit without being sweet and sticky. Nice carmely flavor, somewhat malty and savory. This beer is a nice convorsational sipper that I would gladly purchase again.","We put a slightly American twist on a classic Vienna Lager to make what we’re calling an “Every Day Beer.” The beer has hints of toasted bran, with a touch of malt sweetness balanced by a slightly heavy handed use of delicate noble hops to finish crisp and keep you coming back for more. We hope you just say, “Gimme a Lager.”","""Lager""",3.50,4.00,4.00,3.50,4.00,3.92,vienna lager beer decent overall value get pay pretty dark red copper hue stien nice palet thick creamy feel fairly low carbination intense aromas alcohol ripe fruit without sweet sticky nice carmely flavor somewhat malty savory beer nice convorsational sipper would gladly purchase put slightly american twist classic vienna lager make we’re c

In [166]:
test

,beer_id,style,text,notes,name,look,smell,taste,feel,overall,score,combined_text
1,53167.0,American Stout,"From bomber to pint glass A: Slightly cloudy and murky, with sediment mixing upon pour (not intended though unavoidable). It has the look of ogranic Honey Crisp apple juice. The head is reasonable, 1.75 fingers. The lacing though is serious, and sticks around for the duration. S: Pineapple and tropical fruit. If I close my eyes I can feel the sun on the back of my neck and the sand under my toes. Hints of sweetened coconut milk intensify as it warms, which is utterly fantastic. T: Just what I was expecting - fresh, lavish hops that burst like a grenade in my mouth. I can't begin to tell you how inviting the citrus is. It actually seems as if I've got an umbrella in my drink while sitting on a motorboat in the islands. M: Slightly acidic, though fairly creamy and thick. The carbonation is ever so slightly sharp, but it's not off-putting. The alcohol is nowhere to be found. O: DuganA is not overly cloying like a lot of DIPA's I've had. The aftertaste sticks around, but you certainly won't smell like a brewery all night. DuganA is a fabulous beer. For as delicious as this bomber was, I must say that having it ON TAP is a genuine experience. It is one of the top ten beers I've had in the last year, and I can guarantee you'll thank me for recommending it. If you see the tap handle, look no further and enjoy.",Rich and roasty aromas of bakers chocolate give way to a soft and rounded mouthfeel and flavors of medium roast coffee and burnt caramel. A firm but restrained bitterness layers with the roast for additional depth of flavor. Wonderfully balanced with all of the flavors perfectly... Incorporated.,"""INC.""",3.50,4.50,4.50,4.00,4.50,4.39,american stout bomber pint glass slightly cloudy murky sediment mixing upon pour intended though unavoidable look ogranic honey crisp apple juice head reasonable fingers lacing though serious sticks around duration pineapple tropical fruit close eyes feel sun back neck sand toes hints sweetened coconut milk intensify warms utterly fantastic expecting fresh lavish hops burst like grenade mouth cant begin tell inviting citrus actually seems ive got umbrella drink sitting motorboat islands slightly acidic though fairly creamy thick carbonation ever slightly sharp offputting alcohol nowhere found dugana overly cloying like lot dipas ive aftertaste sticks around certainly wont smell like brewery night dugana fabulous beer delicious bomber must say tap genuine experience one top ten beers ive last year guarantee youll thank recommending see tap handle look enjoy rich roasty aromas bakers chocolate give way soft rounded mouthfeel flavors medium roast coffee burnt caramel firm restrained bitterness layers roast additional depth flavor wonderfully balanced flavors perfectly incorporated
2,52283.0,Vienna Lager,"This beer is a decent overall value being that you get what you pay for. It has a pretty, dark red copper hue in the stien. Nice on the palet with a thick creamy feel. Fairly low on carbination. Intense aromas of alcohol and ripe fruit without being sweet and sticky. Nice carmely flavor, somewhat malty and savory. This beer is a nice convorsational sipper that I would gladly purchase again.","We put a slightly American twist on a classic Vienna Lager to make what we’re calling an “Every Day Beer.” The beer has hints of toasted bran, with a touch of malt sweetness balanced by a slightly heavy handed use of delicate noble hops to finish crisp and keep you coming back for more. We hope you just say, “Gimme a Lager.”","""Lager""",3.50,4.00,4.00,3.50,4.00,3.92,vienna lager beer decent overall value get pay pretty dark red copper hue stien nice palet thick creamy feel fairly low carbination intense aromas alcohol ripe fruit without sweet sticky nice carmely flavor somewhat malty savory beer nice convorsational sipper would gladly purchase put slightly american twist classic vienna lager make we’re c

In [167]:
test_num_df = test[['look', 'smell', 'taste', 'feel', 'overall']]
print(test_num_df)
test_num_array = np.array(test_num_df)
print(test_num_array)
print(len(test_num_array))

       look  smell  taste  feel  overall
1      3.50  4.50   4.50   4.00  4.50   
2      3.50  4.00   4.00   3.50  4.00   
3      4.25  4.00   4.25   4.00  4.25   
4      4.25  3.50   3.75   4.00  4.25   
5      3.50  4.25   4.00   3.75  4.00   
...     ...   ...    ...    ...   ...   
21717  3.50  4.00   4.00   4.00  4.00   
21718  3.00  2.75   2.75   2.75  2.75   
21719  4.50  4.00   4.50   4.00  4.25   
21720  4.50  4.50   4.75   4.25  4.75   
21721  4.00  4.00   4.50   4.00  4.00   

[21721 rows x 5 columns]
[[3.5  4.5  4.5  4.   4.5 ]
 [3.5  4.   4.   3.5  4.  ]
 [4.25 4.   4.25 4.   4.25]
 ...
 [4.5  4.   4.5  4.   4.25]
 [4.5  4.5  4.75 4.25 4.75]
 [4.   4.   4.5  4.   4.  ]]
21721


In [168]:
numerators = np.array([test_num_array[0].dot(beers) for beers in test_num_array[0:]])
denominators = np.array([np.sqrt(sum(test_num_array[0]**2)) *\
                         np.sqrt(sum(beers**2)) for beers in test_num_array[0:]])

test_results = pd.DataFrame((numerators / denominators), columns=['recommendation'])
new_test = test.drop([0])
new_test = new_test.join(results)

KeyError: '[0] not found in axis'

In [ ]:
numerators = np.array([test_num_array[0].dot(beers) for beers in test_num_array[0:]])
denominators = np.array([np.sqrt(sum(test_num_array[0]**2)) *\
                         np.sqrt(sum(beers**2)) for beers in test_num_array[0:]])

test_results = pd.DataFrame((numerators / denominators), columns=['recommendation'])
new_test = new_test.join(results)

In [169]:
numerators.shape

(21721,)

In [170]:
denominators.shape

(21721,)

In [171]:
test_num_array

array([[3.5 , 4.5 , 4.5 , 4.  , 4.5 ],
       [3.5 , 4.  , 4.  , 3.5 , 4.  ],
       [4.25, 4.  , 4.25, 4.  , 4.25],
       ...,
       [4.5 , 4.  , 4.5 , 4.  , 4.25],
       [4.5 , 4.5 , 4.75, 4.25, 4.75],
       [4.  , 4.  , 4.5 , 4.  , 4.  ]])

In [172]:
new_test.isna().sum()

NameError: name 'new_test' is not defined

In [173]:
new_test

NameError: name 'new_test' is not defined

In [174]:
test

,beer_id,style,text,notes,name,look,smell,taste,feel,overall,score,combined_text
1,53167.0,American Stout,"From bomber to pint glass A: Slightly cloudy and murky, with sediment mixing upon pour (not intended though unavoidable). It has the look of ogranic Honey Crisp apple juice. The head is reasonable, 1.75 fingers. The lacing though is serious, and sticks around for the duration. S: Pineapple and tropical fruit. If I close my eyes I can feel the sun on the back of my neck and the sand under my toes. Hints of sweetened coconut milk intensify as it warms, which is utterly fantastic. T: Just what I was expecting - fresh, lavish hops that burst like a grenade in my mouth. I can't begin to tell you how inviting the citrus is. It actually seems as if I've got an umbrella in my drink while sitting on a motorboat in the islands. M: Slightly acidic, though fairly creamy and thick. The carbonation is ever so slightly sharp, but it's not off-putting. The alcohol is nowhere to be found. O: DuganA is not overly cloying like a lot of DIPA's I've had. The aftertaste sticks around, but you certainly won't smell like a brewery all night. DuganA is a fabulous beer. For as delicious as this bomber was, I must say that having it ON TAP is a genuine experience. It is one of the top ten beers I've had in the last year, and I can guarantee you'll thank me for recommending it. If you see the tap handle, look no further and enjoy.",Rich and roasty aromas of bakers chocolate give way to a soft and rounded mouthfeel and flavors of medium roast coffee and burnt caramel. A firm but restrained bitterness layers with the roast for additional depth of flavor. Wonderfully balanced with all of the flavors perfectly... Incorporated.,"""INC.""",3.50,4.50,4.50,4.00,4.50,4.39,american stout bomber pint glass slightly cloudy murky sediment mixing upon pour intended though unavoidable look ogranic honey crisp apple juice head reasonable fingers lacing though serious sticks around duration pineapple tropical fruit close eyes feel sun back neck sand toes hints sweetened coconut milk intensify warms utterly fantastic expecting fresh lavish hops burst like grenade mouth cant begin tell inviting citrus actually seems ive got umbrella drink sitting motorboat islands slightly acidic though fairly creamy thick carbonation ever slightly sharp offputting alcohol nowhere found dugana overly cloying like lot dipas ive aftertaste sticks around certainly wont smell like brewery night dugana fabulous beer delicious bomber must say tap genuine experience one top ten beers ive last year guarantee youll thank recommending see tap handle look enjoy rich roasty aromas bakers chocolate give way soft rounded mouthfeel flavors medium roast coffee burnt caramel firm restrained bitterness layers roast additional depth flavor wonderfully balanced flavors perfectly incorporated
2,52283.0,Vienna Lager,"This beer is a decent overall value being that you get what you pay for. It has a pretty, dark red copper hue in the stien. Nice on the palet with a thick creamy feel. Fairly low on carbination. Intense aromas of alcohol and ripe fruit without being sweet and sticky. Nice carmely flavor, somewhat malty and savory. This beer is a nice convorsational sipper that I would gladly purchase again.","We put a slightly American twist on a classic Vienna Lager to make what we’re calling an “Every Day Beer.” The beer has hints of toasted bran, with a touch of malt sweetness balanced by a slightly heavy handed use of delicate noble hops to finish crisp and keep you coming back for more. We hope you just say, “Gimme a Lager.”","""Lager""",3.50,4.00,4.00,3.50,4.00,3.92,vienna lager beer decent overall value get pay pretty dark red copper hue stien nice palet thick creamy feel fairly low carbination intense aromas alcohol ripe fruit without sweet sticky nice carmely flavor somewhat malty savory beer nice convorsational sipper would gladly purchase put slightly american twist classic vienna lager make we’re c

In [177]:
b = rec_group[rec_group['name'] == 'Bud Light']

In [178]:
b

,beer_id,style,text,notes,name,look,smell,taste,feel,overall,score,combined_text,recommendation
3498,175747.0,American Light Lager,,"Bud Light is brewed using a blend of premium aroma hop varieties, both American-grown and imported, and a combination of barley malts and rice. Its superior drinkability and refreshing flavor makes it the world’s favorite light beer.",Bud Light,4.0,4.25,4.0,4.0,4.0,4.06,american light lager bud light brewed using blend premium aroma hop varieties americangrown imported combination barley malts rice superior drinkability refreshing flavor makes world’s favorite light beer,0.99894


In [179]:
print(b['notes'])

3498    Bud Light is brewed using a blend of premium aroma hop varieties, both American-grown and imported, and a combination of barley malts and rice. Its superior drinkability and refreshing flavor makes it the world’s favorite light beer.
Name: notes, dtype: object


In [180]:
test

,beer_id,style,text,notes,name,look,smell,taste,feel,overall,score,combined_text
1,53167.0,American Stout,"From bomber to pint glass A: Slightly cloudy and murky, with sediment mixing upon pour (not intended though unavoidable). It has the look of ogranic Honey Crisp apple juice. The head is reasonable, 1.75 fingers. The lacing though is serious, and sticks around for the duration. S: Pineapple and tropical fruit. If I close my eyes I can feel the sun on the back of my neck and the sand under my toes. Hints of sweetened coconut milk intensify as it warms, which is utterly fantastic. T: Just what I was expecting - fresh, lavish hops that burst like a grenade in my mouth. I can't begin to tell you how inviting the citrus is. It actually seems as if I've got an umbrella in my drink while sitting on a motorboat in the islands. M: Slightly acidic, though fairly creamy and thick. The carbonation is ever so slightly sharp, but it's not off-putting. The alcohol is nowhere to be found. O: DuganA is not overly cloying like a lot of DIPA's I've had. The aftertaste sticks around, but you certainly won't smell like a brewery all night. DuganA is a fabulous beer. For as delicious as this bomber was, I must say that having it ON TAP is a genuine experience. It is one of the top ten beers I've had in the last year, and I can guarantee you'll thank me for recommending it. If you see the tap handle, look no further and enjoy.",Rich and roasty aromas of bakers chocolate give way to a soft and rounded mouthfeel and flavors of medium roast coffee and burnt caramel. A firm but restrained bitterness layers with the roast for additional depth of flavor. Wonderfully balanced with all of the flavors perfectly... Incorporated.,"""INC.""",3.50,4.50,4.50,4.00,4.50,4.39,american stout bomber pint glass slightly cloudy murky sediment mixing upon pour intended though unavoidable look ogranic honey crisp apple juice head reasonable fingers lacing though serious sticks around duration pineapple tropical fruit close eyes feel sun back neck sand toes hints sweetened coconut milk intensify warms utterly fantastic expecting fresh lavish hops burst like grenade mouth cant begin tell inviting citrus actually seems ive got umbrella drink sitting motorboat islands slightly acidic though fairly creamy thick carbonation ever slightly sharp offputting alcohol nowhere found dugana overly cloying like lot dipas ive aftertaste sticks around certainly wont smell like brewery night dugana fabulous beer delicious bomber must say tap genuine experience one top ten beers ive last year guarantee youll thank recommending see tap handle look enjoy rich roasty aromas bakers chocolate give way soft rounded mouthfeel flavors medium roast coffee burnt caramel firm restrained bitterness layers roast additional depth flavor wonderfully balanced flavors perfectly incorporated
2,52283.0,Vienna Lager,"This beer is a decent overall value being that you get what you pay for. It has a pretty, dark red copper hue in the stien. Nice on the palet with a thick creamy feel. Fairly low on carbination. Intense aromas of alcohol and ripe fruit without being sweet and sticky. Nice carmely flavor, somewhat malty and savory. This beer is a nice convorsational sipper that I would gladly purchase again.","We put a slightly American twist on a classic Vienna Lager to make what we’re calling an “Every Day Beer.” The beer has hints of toasted bran, with a touch of malt sweetness balanced by a slightly heavy handed use of delicate noble hops to finish crisp and keep you coming back for more. We hope you just say, “Gimme a Lager.”","""Lager""",3.50,4.00,4.00,3.50,4.00,3.92,vienna lager beer decent overall value get pay pretty dark red copper hue stien nice palet thick creamy feel fairly low carbination intense aromas alcohol ripe fruit without sweet sticky nice carmely flavor somewhat malty savory beer nice convorsational sipper would gladly purchase put slightly american twist classic vienna lager make we’re c

In [185]:
test.loc()

TypeError: unhashable type: 'list'